# Introduction
This note book creates and saves a dataframe for the MLB hitting stats for the past 5 years.

# Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Notebook Setup

## Work around 403 error

In [2]:
# a small little ruse
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

## Scraping Website

[MLB.com](https://www.mlb.com/stats/)

The base url for regular season is below. Notice the only parts of the url that change are the year and page number.



In [3]:
# https://www.mlb.com/stats/{year}?page={page}&playerPool=ALL

I want scrap 5 years of data. Remember the 2020 season was cancelled due to Covid. This means starting in 2022, I will need to scrap back to 2017. 

# Finding 'stats-body-table player' table from [MLB website](https://www.mlb.com/stats)

# Create Table

In [ ]:
years_to_scrap = ['2022', '2021', '2019', '2018', '2017']

year in years_to_scrap:
    
    # page number for initializing the table
    page = 1
    
    # base URL with year and page variables
    base_url = f'https://www.mlb.com/stats/{year}?page={page}&playerPool=ALL'
    
    # request for beautiful soup
    response = requests.get(base_URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the table from website
    table = soup.find('div', class_ = 'stats-body-table player')

    
    

In [ ]:
    # page number for initializing the table
    i = 1

    # request for beautiful soup
    response = requests.get(base_URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the table from website
    table = soup.find('div', class_ = 'stats-body-table player')

    # convert to pandas df
    stats_table = pd.read_html(str(table))[0]

    # scrap multiple pages
    # start on page 2, first page used to initialize table
    for i in range(2,num_of_pages_to_scrap):

        # request for beautiful soup
        response = requests.get(make_url(url, i), headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # find the table
        table = soup.find('div', class_ = 'stats-body-table player')

        # initialize the table - convert to pandas df
        pandas_table = pd.read_html(str(table))[0]

        # check if there is data in the table
        if len(pandas_table) > 0:

            # if there is info in table then concat it to initialized table
            stats_table = pd.concat([stats_table, pandas_table])

        # if no data in the table then we have reached the end, break out of loop
        else:
            break

    # change column names from NAMENAME to NAME
    fix_col_names = []
    for col in stats_table.columns:
        if 'caret-upcaret-down' in col:
            name = col.split('caret-upcaret-down')[-1]
        else:
            name = col[:int(len(col)/2)]
        fix_col_names.append(name)

    stats_table.columns = fix_col_names

    # reset the index, drop old one
    stats_table.reset_index(drop=True, inplace=True)      

    return stats_table